In [1]:
# # --------------------------------------------------
# # 1️⃣ Mount Google Drive (optional, for cache)
# # --------------------------------------------------
# from google.colab import drive
# import os

# if not os.path.ismount("/content/drive"):
#     drive.mount("/content/drive")
# else:
#     print("📦 Google Drive already mounted")

# --------------------------------------------------
# 2️⃣ Clone fenicsx-colab repository (idempotent)
# --------------------------------------------------
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

def run(cmd):
    #print("$", " ".join(map(str, cmd)))
    subprocess.run(cmd, check=True)

if not REPO_DIR.exists():
    print("📥 Cloning fenicsx-colab...")
    run(["git", "clone", REPO_URL, str(REPO_DIR)])
elif not (REPO_DIR / ".git").exists():
    raise RuntimeError("Directory exists but is not a git repository")
else:
    print("📦 Repository already exists — skipping clone")

# --------------------------------------------------
# 3️⃣ Run setup_fenicsx.py IN THIS KERNEL (CRITICAL)
# --------------------------------------------------
print("🚀 Running setup_fenicsx.py in current kernel")

USE_CLEAN = False  # <--- Set True to remove existing environment
opts = "--clean" if USE_CLEAN else ""

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts}"
)

# ==================================================
# 4️⃣ Sanity check
# ==================================================
try:
    get_ipython().run_cell_magic('fenicsx', '--info -np 4', '')
except Exception as e:
    print("⚠️ %%fenicsx magic not found:", e)

📥 Cloning fenicsx-colab...
🚀 Running setup_fenicsx.py in current kernel
⚠️ Google Drive not mounted — using local cache (/content)
🔧 Installing FEniCSx environment...
✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

🐍 Python          : 3.11.14
📦 dolfinx         : 0.10.0
💻 Platform        : Linux-6.6.105+-x86_64-with-glibc2.35
🧵 Running as root : True

🔎 fenicsx runtime info
-----------------------
Environment        : fenicsx
micromamba         : /content/micromamba/bin/micromamba
MPI implementation : OPENMPI
MPI version        : mpiexec (OpenRTE) 4.1.2
MPI ranks (-np)    : 4


# Poisson 2D Complete Example (FEniCSx)

This notebook solves the Poisson equation on the unit square using FEniCSx and MPI.


## Problem description

We solve

$$
\begin{aligned}
  -\Delta u &= 2\pi^2 \sin\pi x \sin\pi y &&\text{in } \Omega=(0,1)^2 \\
  u &=0 &&\text{on } \partial\Omega.
\end{aligned}
$$

오메가: 0,1의 정사각형 영역. 그 경계에서 u가 0이라는 뜻

- Finite elements: CG1 (P1)
- Parallel execution using MPI


In [2]:
%%fenicsx -np 4 --time

import numpy as np
from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import mesh, fem, io
from dolfinx.fem.petsc import LinearProblem
import ufl

# --------------------------------------------------
# Build mesh + solve
# --------------------------------------------------

def solve_poisson(comm, nx=32, ny=32):
    # Mesh & function space
    domain = mesh.create_unit_square(comm, nx, ny)
    x = ufl.SpatialCoordinate(domain)

    # Exact solution and RHS
    u_exact = ufl.sin(np.pi *x[0]) *ufl.sin(np.pi *x[1])
    f = 2.0 *np.pi**2 *u_exact

    V = fem.functionspace(domain, ("Lagrange", 1))
    # Dirichlet BC: u = 0 on boundary
    u_bc = fem.Function(V)
    u_bc.x.array[:] = 0.0

    fdim = domain.topology.dim -1
    boundary_facets = mesh.locate_entities_boundary(
        domain, fdim,
        lambda x: np.full(x.shape[1], True)
    )
    bc = fem.dirichletbc(
        u_bc,
        fem.locate_dofs_topological(V, fdim, boundary_facets)
    )

    # Variational problem
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)

    a = ufl.inner(ufl.grad(u), ufl.grad(v)) *ufl.dx
    L = f *v *ufl.dx

    problem = LinearProblem(
        a, L, bcs=[bc],
        petsc_options={"ksp_type": "cg", "pc_type": "hypre"},
        petsc_options_prefix="poisson_",
    )

    uh = problem.solve()

    # Diagnostics
    local_L2 = fem.assemble_scalar(
        fem.form((uh -u_exact) *(uh -u_exact) *ufl.dx)
    )
    global_L2 = comm.allreduce(local_L2, op=MPI.SUM)

    if comm.rank == 0:
        print("✅ Poisson problem solved")
        print("   MPI size      :", comm.size)
        print("   Number of dofs:", V.dofmap.index_map.size_global)
        print("   L2 norm       :", np.sqrt(global_L2))

    return domain, uh

# --------------------------------------------------
# save_xdmf: collective I/O
# --------------------------------------------------

def save_xdmf(comm, domain, uh, filename="poisson.xdmf"):

    with io.XDMFFile(comm, filename, "w") as xdmf:
        xdmf.write_mesh(domain)
        xdmf.write_function(uh)

    if comm.rank == 0:
        print(f"🖼️ Saved {filename}\n")

# --------------------------------------------------

comm = MPI.COMM_WORLD

domain, uh = solve_poisson(comm)
save_xdmf(comm, domain, uh)

✅ Poisson problem solved
   MPI size      : 4
   Number of dofs: 1089
   L2 norm       : 0.0013504333788826707
🖼️ Saved poisson.xdmf

⏱ Elapsed time: 8.244993 s


In [3]:
from google.colab import files

files.download("poisson.xdmf")
files.download("poisson.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>